## Lecture 10

## Python Parallel Computing - Part 01

### Apr 07, 2020


Part of this lecture is based on the previous years lectures and the material [parallel computing](https://nyu-cds.github.io/python-mpi/01-introduction/).


---

You will need **mpi4py** 

To install: [https://mpi4py.readthedocs.io/en/stable/install.html](https://mpi4py.readthedocs.io/en/stable/install.html)


Run the following in terminal: 

1. sudo apt install libopenmpi-dev

2. pip install mpi4py


or 

1. brew install open-mpi

2. pip3 install mpi4py


If errors, see this article for further details: [https://stackoverflow.com/questions/28440834/error-when-installing-mpi4py](https://stackoverflow.com/questions/28440834/error-when-installing-mpi4py)

---


In [17]:
conda install mpi4py

Solving environment: done

## Package Plan ##

  environment location: /Users/chengweichen/anaconda3

  added / updated specs:
    - mpi4py


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.1.1   |                0         132 KB
    certifi-2020.4.5.1         |           py37_0         159 KB
    conda-4.8.3                |           py37_0         3.0 MB
    mpi-1.0                    |            mpich          13 KB
    mpi4py-3.0.3               |   py37h77202c6_0         563 KB
    mpich-3.3.2                |       hc856adb_0         4.1 MB
    openssl-1.1.1f             |       h1de35cc_0         3.5 MB
    ------------------------------------------------------------
                                           Total:        11.5 MB

The following NEW packages will be INSTALLED:

  mpi                pkgs/main/osx-64::mpi-1.0-mpich
  mpi4py             pkgs/ma

### Two basic approaches

<img src="./figs/shared_memory.png" alt="shared_memory" style="width: 350px;"/>


<img src="./figs/distributed_memory.png" alt="distributed_memory" style="width: 400px;"/>

---

### __MPI__ (Message Passing Interface) 

- Most widely used standard


- For programming distributed-memory multiple instruction--multiple data (MIMD) systems


#### __Point to point Communication__

Processes should coordinate their activities by explicitly sending and receiving messages

MPI operates as follows:
- Process A decides a message needs to be sent to process B.
- Process A packs up all of its necessary data into a buffer for process B.
- Process A indicates that the data should be sent to process B by calling the _Send_ function.
- Process B needs to acknowledge it wants to receive the message by calling the _Recv_ function.

Every time a process sends a message, there must be a process that also indicates it wants to receive the message, therefore, calls to _Send_ and _Recv_ are always paired.



<img src="./figs/send_receive.png" alt="distributed_memory" style="width: 400px;"/>

---

### The number of processes 

- Is **fixed** when an MPI program is started 

- Each of the processes is assigned a unique integer starting from 0. 

- This integer is know as the rank of the process and is how each process is identified when sending and receiving messages (we will refer to rank K process as "process K").

- **MPI processes** are arranged in logical collections known as **communicators**. 

- There is one special communicator (**MPI.COMM_WORLD**) that exists when an MPI program starts, which contains all the processes in the MPI program. 


- MPI provides a few **methods** on a communicator:


> Get_size() - returns the total number of processes contained in the communicator (the size of the communicator).

> Get_rank() - returns the rank of the calling process within the communicator. 

> Send() - sends content to a process

> Recv() - receives content from a process



In [3]:
from mpi4py import MPI

comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()
print('hello world: size = %d, rank = %d' % (size, rank))

hello world: size = 1, rank = 0


In [22]:
%%writefile mpi1.py
#####
# writting the code in the mpi1.py file
#####

from mpi4py import MPI

comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()
print('hello world: size = %d, rank = %d' % (size, rank))

Overwriting mpi1.py


In [23]:
#####
# running MPI from the terminal with n=4 processes
#####
!mpiexec -n 2 python3 mpi1.py

hello world: size = 2, rank = 1
hello world: size = 2, rank = 0


---
### One MPI program, multiple MPI processes
Making each process to perform a different computation 

In [6]:
%%writefile mpi2.py

from mpi4py import MPI
rank = MPI.COMM_WORLD.Get_rank()

a = 8.0
b = 4.0
print('Process rank',rank)
if rank == 0:
        print("addition:", a + b)
if rank == 1:
        print("multiplication:", a * b)
if rank == 2:
        print("maximum:", max(a,b))
if rank == 3:
        print("doing nothing:")

Overwriting mpi2.py


In [7]:
!mpiexec -n 4 python3 mpi2.py

Process rank 2
maximum: 8.0
Process rank 3
doing nothing:
Process rank 0
addition: 12.0
Process rank 1
multiplication: 32.0


---
### Point-to-point communication
Message passing involves two processes: a **sender** and a **receiver** (commands _Send_ and _Recv_).

In [8]:
%%writefile mpi3.py
#####
# Sending a message from one process to another
#####
import numpy

from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()

randNum = numpy.zeros(1)

if rank == 1:
        # generates a numpy array with one element unif. distr. from [0,1)
        randNum = numpy.random.random_sample(1)
        print("part of Process", rank, "- drew the number", randNum[0])
        comm.Send(randNum, dest=0)
        
if rank == 0:
        print("part of Process", rank, "- before receiving has the number", randNum[0])
        comm.Recv(randNum, source=1)
        print("part of Process", rank, "- received the number", randNum[0])

Overwriting mpi3.py


In [9]:
!mpiexec -n 2 python3 mpi3.py

part of Process 1 - drew the number 0.7633146341149917
part of Process 0 - before receiving has the number 0.0
part of Process 0 - received the number 0.7633146341149917


In [10]:
%%writefile mpi4.py
#####
# Sending a message to a process and receiving a message back
#####

import numpy
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()

randNum = numpy.zeros(1) 

if rank == 1:
        randNum = numpy.random.random_sample(1)
        print("Process", rank, "drew the number", randNum[0])
        comm.Send(randNum, dest=0)
        comm.Recv(randNum, source=0)
        print("Process", rank, "received the number", randNum[0], "from process 0")
        
if rank == 0:
        print("Process", rank, "before receiving has the number", randNum[0])
        comm.Recv(randNum, source=1)
        print("Process", rank, "received the number", randNum[0], "from process 1")
        randNum *= 2
        comm.Send(randNum, dest=1) 

Overwriting mpi4.py


<img src="./figs/send_receive_mul2.png" style="width: 400px;"/>

In [11]:
!mpiexec -n 2 python3 mpi4.py

Process 0 before receiving has the number 0.0
Process 0 received the number 0.029037491033082508 from process 1
Process 1 drew the number 0.029037491033082508
Process 1 received the number 0.058074982066165015 from process 0


---

The receiving process does not always need to specify the source when issuing a Recv. Instead, the process can accept **any message** that is being sent by another process. This is done by setting the source to **MPI.ANY_SOURCE**.

In [12]:
%%writefile mpi5.py
#####
# Sending a message to a process and receiving a message back from MPI.ANY_SOURCE
#####

import numpy
from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

randNum = numpy.zeros(1) 

if rank == 1:
        randNum = numpy.random.random_sample(1)
        print("Process", rank, "drew the number", randNum[0])
        comm.Send(randNum, dest=0)
        comm.Recv(randNum, source=MPI.ANY_SOURCE)
        print("Process", rank, "received the number", randNum[0], "from process", MPI.ANY_SOURCE)
        
if rank == 0:
        print("Process", rank, "before receiving has the number", randNum[0])
        comm.Recv(randNum, source=MPI.ANY_SOURCE)
        print("Process", rank, "received the number", randNum[0])    
        randNum *= 2
        comm.Send(randNum, dest=1)

Overwriting mpi5.py


In [13]:
!mpiexec -n 2 python3 mpi5.py

Process 0 before receiving has the number 0.0
Process 0 received the number 0.6261875244338339
Process 1 drew the number 0.6261875244338339
Process 1 received the number 1.2523750488676677 from process -2


---

Sometimes there are cases when a process might have to **send many different types of messages to another process**. Instead of having to go through extra measures to differentiate all these messages, MPI allows senders and receivers to also **specify message IDs (known as tags)** with the message. The receiving process can then request a message with a certain tag number and messages with different tags will be buffered until the process requests them.

```python
Comm.Send(buf, dest=0, tag=0)
Comm.Recv(buf, source=0, tag=0, status=None)
```

The _status_ can provide useful information
```python
info = MPI.Status()
source = info.Get_source()
tag = info.Get_tag()
count = info.Get_elements()
size = info.Get_count()
```

In [14]:
%%writefile mpi_tag.py
from mpi4py import MPI

comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank() 

data1 = None
data2 = None

if rank == 0:
    data1 = ('a','b', 'c', 'd')
    data2 = (1, 2, 3, 4)
    comm.send(data1, dest=1, tag=100)
    comm.send(data2, dest=1, tag=200)

elif rank == 1:
    print('On Process',rank,'before recv: data1 = ', data1)
    print('On Process',rank,'before recv: data2 = ', data2)
    
    data1 = comm.recv(source=0, tag=100)
    data2 = comm.recv(source=0, tag=200)
    
    print('On Process',rank,'after  recv: data1 = ', data1)
    print('On Process',rank,'after  recv: data2 = ', data2)
    

Overwriting mpi_tag.py


In [15]:
!mpiexec -n 2 python3 mpi_tag.py

On Process 1 before recv: data1 =  None
On Process 1 before recv: data2 =  None
On Process 1 after  recv: data1 =  ('a', 'b', 'c', 'd')
On Process 1 after  recv: data2 =  (1, 2, 3, 4)


In [16]:
%%writefile mpi_status.py
#####
# Sending a message from one process to another
#####


import numpy
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()

info = MPI.Status()
# print("info: ", info)

randNum = numpy.zeros(1)

if rank == 1:
        randNum = numpy.random.random_sample(1)
        print("Process", rank, "drew the number", randNum[0])
        comm.Send(randNum, dest=0)

if rank == 0:
        print("Process", rank, "before receiving has the number", randNum[0])
        comm.Recv(randNum, source=1, status=info)
        print("Process", rank, "received the number", randNum[0], "from Process", info.Get_source())

Overwriting mpi_status.py


In [17]:
!mpiexec -n 2 python3 mpi_status.py

Process 1 drew the number 0.5441525081188491
Process 0 before receiving has the number 0.0
Process 0 received the number 0.5441525081188491 from Process 1


---
### Non-blocking Communication

In the previous examples, the sender and receiver are not able to perform any action when sending or receiving a message, wasting computational times while waiting for the call to complete. 

__Non-blocking communcation__ avoids this issue by using the _Isend_ and _Irecv_ methods, which start to send and receive operations and _then return immediately to continue computation_.

The completion of a send or receive operation can be managed using the _Test_, _Wait_, and _Cancel_ methods.


In [18]:
%%writefile mpi6.py
#####
# this code is similar to mpi3.py, 
# but it uses Wait to block the processes
#####

import numpy
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()

randNum = numpy.zeros(1)

if rank == 1:
        randNum = numpy.random.random_sample(1)
        print("Process", rank, "drew the number", randNum[0])
        
        req = comm.Isend(randNum, dest=0)
#         req.Wait()
        
        print('something here')
        
if rank == 0:
        print("Process", rank, "before receiving has the number", randNum[0])
        
        req = comm.Irecv(randNum, source=1)
        req.Wait()
        
        print("Process", rank, "received the number", randNum[0])

Overwriting mpi6.py


In [19]:
!mpiexec -n 2 python3 mpi6.py

Process 1 drew the number 0.3059253451245264
something here
Process 0 before receiving has the number 0.0
Process 0 received the number 0.3059253451245264


### Overlap communication


**Example:** Process 1 overlaps a computation with sending the message and receiving the reply. The computation divides randNum by 10 and prints the result.

In [20]:
%%writefile mpi7.py
#####
# overlap communication
#####

import numpy
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()

randNum = numpy.zeros(1) 

if rank == 1:
        randNum = numpy.random.random_sample(1)
        print("Process", rank, "drew the number", randNum[0])
        comm.Isend(randNum, dest=0)
        randNum[0] /= 10 # overlap communication
        print("Process", rank, "number in overlap communication =", randNum[0])
        req = comm.Irecv(randNum, source=0)
        req.Wait()
        print("Process", rank, "received the number", randNum[0])

if rank == 0:
        print("Process", rank, "before receiving has the number", randNum[0])
        req = comm.Irecv(randNum, source=1)
        req.Wait()
        print("Process", rank, "received the number", randNum[0])
        randNum *= 2
        comm.Isend(randNum, dest=1)

Overwriting mpi7.py


In [21]:
!mpiexec -n 2 python3 mpi7.py

Process 0 before receiving has the number 0.0
Process 0 received the number 0.8794599647125977
Process 1 drew the number 0.8794599647125977
Process 1 number in overlap communication = 0.08794599647125977
Process 1 received the number 1.7589199294251954
